In [1]:
import cv2
import numpy as np
import os
import time
import datetime
import mediapipe as mp

from matplotlib import pyplot as plt
from statistics import mode
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GaussianNoise
from tensorflow.keras.callbacks import TensorBoard

In [2]:
# Diretório para os pontos coletados
DATA_DIR = os.path.join('Dados_extraidos') 

# Sinais
sinais = np.array(['Ola','Tudo bem','Obrigado','Entendo','Calma'])

# Quantidade de vídeos por sinal
num_videos = 400

# Quantidade de frames por vídeo
num_frames = 30

#Quantidade de coordenadas por frame
num_coordenadas = 1662

# Quantidade de frames por vídeo
num_frames = 30

In [3]:
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
cam = cv2.VideoCapture(0)

In [4]:
def mediapipe_extrai(imagem, modelo):
    imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB) 
    imagem.flags.writeable = False                  
    resultados = modelo.process(imagem)                
    imagem.flags.writeable = True                   
    imagem = cv2.cvtColor(imagem, cv2.COLOR_RGB2BGR) 
    return imagem, resultados

In [5]:
def mediapipe_desenha(imagem, resultados):
    # Pontos do rosto
    mp_drawing.draw_landmarks(imagem, resultados.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(153,0,153), thickness=1, circle_radius=1), 
                             mp_drawing_styles.get_default_face_mesh_contours_style()
                             ) 
    # Pontos do corpo
    mp_drawing.draw_landmarks(imagem, resultados.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(153,0,153), thickness=2, circle_radius=3), 
                             mp_drawing.DrawingSpec(color=(255,40,130), thickness=2, circle_radius=1)
                             ) 
    # Pontos da mão esquerda
    mp_drawing.draw_landmarks(imagem, resultados.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(153,0,153), thickness=1, circle_radius=3), 
                             mp_drawing.DrawingSpec(color=(35,100,255), thickness=1, circle_radius=1)
                             ) 
    # Pontos da mão direita  
    mp_drawing.draw_landmarks(imagem, resultados.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(153,0,153), thickness=1, circle_radius=3), 
                             mp_drawing.DrawingSpec(color=(247,0,247), thickness=1, circle_radius=1)
                             ) 

In [6]:
def mediapipe_desenha_extrai(cam, modelo):
    ret, frame = cam.read()                             # Ler o frame
    imagem, resultados = mediapipe_extrai(frame, modelo)# Fazer detecções
    mediapipe_desenha(imagem, resultados)                # Desenhar pontos
    return imagem, resultados

In [7]:
def trata_pontos(resultados):

    rosto = []
    corpo = []
    mao_esquerda = []
    mao_direita = []

    if resultados.face_landmarks:
        for r in resultados.face_landmarks.landmark:
            ponto = np.array([r.x, r.y, r.z])
            rosto.append(ponto)
    else:
        rosto = np.zeros(468*3) 

    if resultados.pose_landmarks:
        for r in resultados.pose_landmarks.landmark:
            ponto = np.array([r.x, r.y, r.z, r.visibility])
            corpo.append(ponto)
    else:
        corpo = np.zeros(33*4) 
    
    if resultados.left_hand_landmarks:
        for r in resultados.left_hand_landmarks.landmark:
            ponto = np.array([r.x, r.y, r.z])
            mao_esquerda.append(ponto)
    else:
        mao_esquerda = np.zeros(21*3)   

    if resultados.right_hand_landmarks:
        for r in resultados.right_hand_landmarks.landmark:
            ponto = np.array([r.x, r.y, r.z])
            mao_direita.append(ponto)
    else:
        mao_direita = np.zeros(21*3)

    rosto = np.array(rosto).flatten() 
    corpo = np.array(corpo).flatten()
    mao_esquerda = np.array(mao_esquerda).flatten()
    mao_direita = np.array(mao_direita).flatten()

    return np.concatenate([corpo, rosto, mao_esquerda, mao_direita])    

In [8]:
# Configura a rede neural

model = Sequential()
model.add(LSTM(512, activation='relu', input_shape=(num_frames,num_coordenadas)))
model.add(Dense(512, activation='relu'))
model.add(Dense(sinais.shape[0], activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.load_weights('pesosbons.h5')

In [10]:
sequencia = []
predicao = ''
limiar_predicao = 0.5
contador_neutro = 0
tempo_inativo = 45
cam = cv2.VideoCapture(0)

with mp_holistic.Holistic() as holistic:
    
    while cam.isOpened():

        # Desenha pontos na imagem e extrai características em resultados
        imagem, resultados = mediapipe_desenha_extrai(cam, holistic)

        # Começa a predição caso saia da posição neutra
        if resultados.left_hand_landmarks or resultados.right_hand_landmarks:
            
            # Trata os pontos e monta a sequencia
            pontos_tratados = trata_pontos(resultados)
            sequencia.append(pontos_tratados)

            # Envia para o modelo preditivo quando possuir num_frame frames
            if len(sequencia) == num_frames:
                sequencia_dimensionada = np.expand_dims(sequencia, axis=0)
                sequencia.clear()
                resultados = model.predict(sequencia_dimensionada)[0]
                maior_indice = np.argmax(resultados)

                # Controle para evitar falso-positivos
                if resultados[maior_indice] > limiar_predicao:
                    predicao = sinais[maior_indice]

            contador_neutro = 0

        else:
            contador_neutro +=1
            sequencia.clear()
        
        # Caso não haja mais predições, apaga a última da tela
        if contador_neutro > tempo_inativo:
            predicao = ''

        # Mostra na tela
        cv2.putText(imagem,'{}'.format(predicao), (20, 40),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(0, 255, 255), thickness = 2)
        cv2.imshow('LIBRAS', imagem)

        # Espera pela tecla
        if cv2.waitKey(3) & 0xFF == ord(' '):
            break

cam.release()
cv2.destroyAllWindows()